# Modeling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from IPython import display
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, RepeatedKFold
from sklearn.metrics import r2_score, mean_squared_error as mse, mean_absolute_error as mae, SCORERS
from sklearn.dummy import DummyRegressor
from catboost import Pool, CatBoostRegressor, cv
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from hyperopt import fmin, tpe, hp, Trials
from sklearn.inspection import permutation_importance

In [2]:
#load dataframe OHE
#parse datetime column
df_ohe = pd.read_csv('ohe_data.csv', parse_dates=['date'])
df_ohe.drop(columns='Unnamed: 0', inplace=True)
df_ohe.set_index('date', inplace=True)

In [3]:
df_ohe.head()

,Avg_Temp(F),Conf_Cases,day_of_week,day_of_year,Year,Month,Day,new_case_percent_pop*,state_id_CT,state_id_MA,state_id_ME,state_id_VT
date,,,,,,,,,,,,
2020-03-01,26.42,1.0,6,61,2020,3,1,0.014225,0,1,0,0
2020-03-02,36.50,1.0,0,62,2020,3,2,0.014225,0,1,0,0
2020-03-03,55.94,1.0,1,63,2020,3,3,0.014225,0,1,0,0
2020-03-04,46.94,2.0,2,64,2020,3,4,0.028450,0,1,0,0
2020-03-05,42.98,8.0,3,65,2020,3,5,0.113799,0,1,0,0


## Train-test split

Before training any models on the data, let's do a train-test split to keep training and testing data consistent and separated.

In [4]:
len(df_ohe)

2224

In [5]:
bins = np.linspace(0, 2224, 200)

In [6]:
# Create features
X, y = df_ohe.drop(columns=['Conf_Cases', 'new_case_percent_pop*']), df_ohe['new_case_percent_pop*']

In [7]:
y_binned = np.digitize(y, bins)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y_binned, random_state=42)

## Using the mean as a baseline prediction model

Previously, we determined the R2 score of using the mean to predict COVID19 cases for each individual state. Let's do the same thing now that we have all the states in one DataFrame, so that we will have a baseline "dummy" model to compare our future optimized models to. 

In [9]:
dummy_mean = DummyRegressor()
# "Train" dummy regressor
dummy_mean.fit(X_train, y_train)
# Get R2 score
score_dummy = dummy_mean.score(X_test, y_test)
print("The R2 score of using the mean to predict COVID19 cases in our states is:", score_dummy)

The R2 score of using the mean to predict COVID19 cases in our states is: -0.0008611149015183006


Let's store the evaluation metric values for the dummy regressor so we can compare them with our future models. 

In [10]:
dummy_pred = dummy_mean.predict(X_test)
dummy_r2 = r2_score(y_test, dummy_pred)
dummy_mse = mse(y_test, dummy_pred)
dummy_rmse = np.sqrt(mse(y_test, dummy_pred))

## Tuning the top performing models for ensemble model

In the pre-processing step, we determined (with the help of Pycaret) that our top performing models were **CatBoost Regressor**, **Random Forest Regressor**, and **Extra Trees Regressor**. Let's now fine tune the hyperparameters of each of these models, using Coarse to Fine hyperparameter tuning, as well as Bayesian Optimization (where applicable), in preparation for feeding them into the pipeline of the Voting Regressor. 

### 1. Random Forest Regressor
With a few exceptions, a RandomForestClassifier has all the hyperparameters of a DecisionTreeClassifier (to control how trees are grown), plus all the hyperparameters of a BaggingClassifier to control the ensemble itself. I'll explore a couple different ways to determine optimal hyperparameters and will choose those that produce the best R2 and MSE for the final ensemble model (a VotingRegressor).

First, let's run a basic hyperparameter grid search and check the resulting R2, MSE, and RMSE. Then we'll put these values in a DataFrame with the those obtained from the dummy regressor (above) and compare the results of each set of hyperparameters determined throughout the course of the Coarse to Fine hyperparameter tuning process.

In [11]:
# Define parameters to search for GridSearchCV
basic_param_grid = {'n_estimators': [100, 300, 500, 900, 1200],
              'max_depth': [3, 5, 20, 50, 100],
              }
# Instantiate RandomForestRegressor
basic_rf = RandomForestRegressor(random_state=42)
cv_rf = GridSearchCV(basic_rf, basic_param_grid, cv = 5)
cv_rf_fit = cv_rf.fit(X_train, y_train)

In [12]:
print('The optimal max_depth for the RandomForestRegressor is: {}'.format(cv_rf_fit.best_params_['max_depth']))
print('The optimal n_estimators for the RandomForestRegressor is: {}'.format(cv_rf_fit.best_params_['n_estimators']))

The optimal max_depth for the RandomForestRegressor is: 50
The optimal n_estimators for the RandomForestRegressor is: 300


In [13]:
# Instantiate RFR with optimal hyperparameters
basic_rf = RandomForestRegressor(max_depth = 50, n_estimators = 300, random_state=42)
basic_rf.fit(X_train, y_train)
basic_rf_pred = basic_rf.predict(X_test)

In [14]:
# Calculate evaluation metrics on basic_rf
basic_rf_r2 = r2_score(y_test, basic_rf_pred)
basic_rf_mse = mse(y_test, basic_rf_pred)
basic_rf_rmse = np.sqrt(basic_rf_mse)

In [15]:
# Create RF results dataframe
rf_results = pd.DataFrame({'Model':['dummy_reg', 'basic_rf'], 'R2': [dummy_r2, basic_rf_r2], 'MSE':[dummy_mse, basic_rf_mse], 'RMSE':[dummy_rmse, basic_rf_rmse]})

In [16]:
rf_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_rf,0.849384,42.810904,6.543004


***

#### Using RandomSearchCV to find optimal hyperparameter values
Now let's perform RandomizedSearchCV across a broader range of hyperparameters (and values) to get an idea of where to focus a grid search of a more comprehensive range of hyperparamters than those evaluated in the basic grid search above.

In [17]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(20, 120, num = 11)]
max_depth.append(None)
#max_depth.append(9)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [18]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, n_jobs=-1, random_state=42)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.9min finished


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [20, 30, 40, 50, 60, 70,
                                                      80, 90, 100, 110, 120,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4, 10],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, verbose=2)

In [19]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 110,
 'bootstrap': True}

In [20]:
# Instantiate RFR with optimal hyperparameters
rf2 = RandomForestRegressor(n_estimators= 900,
                             min_samples_split= 2,
                             min_samples_leaf= 2,
                             max_features= 'auto',
                             max_depth= 90,
                             bootstrap= True, 
                             random_state=42)
rf2.fit(X_train, y_train)
rf_pred2 = rf2.predict(X_test)

In [21]:
# Determine R2, MSE, RMSE 
rf_random_r2 = r2_score(y_test, rf_pred2)
rf_random_mse = mse(y_test, rf_pred2)
rf_random_rmse = np.sqrt(mse(y_test, rf_pred2))

In [22]:
# Add results to rf_results df
rf_results.loc[len(rf_results)]= ['rf_randomCV', 
                                 r2_score(y_test, rf_pred2),
                                 mse(y_test, rf_pred2),
                                 np.sqrt(mse(y_test, rf_pred2))]

In [23]:
rf_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_rf,0.849384,42.810904,6.543004
2,rf_randomCV,0.849289,42.838057,6.545079


***

### Using GridSearchCV to find the optimal hyperparameter values
We can now use the insights we gained from the above random search to perform a grid search across a broader range of hyperparameters (with more focused values).

In [ ]:
# Create the parameter grid based on the results of random search 
GS_param_grid = {
    'bootstrap': [True],
    'max_depth': [70, 80, 90, 100, 110],
    'max_features': ['auto'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 3],
    'n_estimators': [700, 800, 900, 1000, 1100]
}

In [ ]:
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = GS_param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
# Instantiate RFR with optimal hyperparameters
rf3 = RandomForestRegressor(n_estimators= 1100,
                             min_samples_split= 2,
                             min_samples_leaf= 2,
                             max_features= 'auto',
                             max_depth= 90,
                             bootstrap= True)
rf3.fit(X_train, y_train)
rf_pred3 = rf3.predict(X_test)

In [ ]:
# Determine R2 score
print('The R2 score for the RandomForestRegressor with GSCV tuning is {}'.format(r2_score(y_test, rf_pred3)))

In [ ]:
print('The MSE of the RandomForestRegressor with GSCV tuning is {}'.format(mse(y_test, rf_pred3)))

In [ ]:
# Add the results to rf_results
rf_results.loc[len(rf_results)]= ['rf_gridCV', 
                                 r2_score(y_test, rf_pred3),
                                 mse(y_test, rf_pred3),
                                 np.sqrt(mse(y_test, rf_pred3))]

In [ ]:
rf_results

### Bayesian Optimization with hyperopt
Lastly, we'll try a different method of determining the best hyperparameter values: Bayesian Optimization. We'll use the hyperopt package across a narrower range of hyperparameters for the sake of time.

In [ ]:
#pip install hyperopt

In [24]:
trials = Trials()

In [25]:
n_estimators = np.arange(50, 1510, 10)
max_depth = np.arange(5, 205, 5)

In [26]:
n_estimators

array([  50,   60,   70,   80,   90,  100,  110,  120,  130,  140,  150,
        160,  170,  180,  190,  200,  210,  220,  230,  240,  250,  260,
        270,  280,  290,  300,  310,  320,  330,  340,  350,  360,  370,
        380,  390,  400,  410,  420,  430,  440,  450,  460,  470,  480,
        490,  500,  510,  520,  530,  540,  550,  560,  570,  580,  590,
        600,  610,  620,  630,  640,  650,  660,  670,  680,  690,  700,
        710,  720,  730,  740,  750,  760,  770,  780,  790,  800,  810,
        820,  830,  840,  850,  860,  870,  880,  890,  900,  910,  920,
        930,  940,  950,  960,  970,  980,  990, 1000, 1010, 1020, 1030,
       1040, 1050, 1060, 1070, 1080, 1090, 1100, 1110, 1120, 1130, 1140,
       1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250,
       1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1340, 1350, 1360,
       1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470,
       1480, 1490, 1500])

In [27]:
max_depth

array([  5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,
        70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125, 130,
       135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195,
       200])

In [28]:
space = {
    "n_estimators": hp.choice("n_estimators", n_estimators),
    "max_depth": hp.choice("max_depth", max_depth),
}

n_estimators-> start from 50 until 1500, 10
max_depth-> start from 5 until 150 or 200 append None
If better hp, experiment with hp for rf and et

In [29]:
# define objective function

def hyperparameter_tuning(params):
    rf_reg = RandomForestRegressor(**params,n_jobs=-1)
    r2 = cross_val_score(rf_reg, X_train, y_train ,scoring="r2").mean()
    return -r2

In [30]:
best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=1000, 
    trials=trials
)

print("Best: {}".format(best))

100%|██████████| 1000/1000 [1:18:34<00:00,  4.71s/trial, best loss: -0.8305086670912729]
Best: {'max_depth': 18, 'n_estimators': 6}


'max_depth': 20, 'n_estimators': 0

In [34]:
# Instantiate RFR with optimal hyperparameters
rf4 = RandomForestRegressor(n_estimators= 105,
                             max_depth= 95)
                             
rf4.fit(X_train, y_train)
rf_pred4 = rf4.predict(X_test)

In [35]:
# Add results to rf_results
rf_results.loc[len(rf_results)]= ['rf_bayes_hyperopt', 
                                 r2_score(y_test, rf_pred4),
                                 mse(y_test, rf_pred4),
                                 np.sqrt(mse(y_test, rf_pred4))]

In [39]:
rf_results = rf_results[:3]

In [40]:
rf_results

,Model,R2,MSE,RMSE
0,dummy_reg,-0.000861,284.483911,16.866651
1,basic_rf,0.849384,42.810904,6.543004
2,rf_randomCV,0.849289,42.838057,6.545079


$\star$ All of our Random Forest Regressors performed remarkably similarly. However, it looks like the optimal hyperparameters chosen by our original basic grid search and our random search performed the best, both in terms of the R2 and mean squared error. Because we tested a wider range of hyperparameters in the random search, we'll use those hyperparameters (as defined in `rf_randomCV`) in our final Voting Regressor ensemble.

***

## 2. Extra Trees Regressor
Now let's determine some optimal hyperparameters for the second estimator in our Voting Regressor: an Extra Trees Regressor. We'll repeat the same process we used for our Random Forest Regressor above, using Coarse to Fine hyperparameter tuning, as well as experimenting with Bayesian Optimization. We'll put the results of each of these searches in a DataFrame for easy comparison, and select the best set of hyperparameters. 

### Using basic hyperparameter tuning only

In [ ]:
# Define parameters to search for GridSearchCV
param_grid = {'n_estimators': [100, 300, 500, 900, 1200],
              'max_depth': [3, 5, 20, 50, 100],
              }
# Instantiate ExtraTreesRegressor
et = ExtraTreesRegressor(random_state=42)
cv_et = GridSearchCV(et, param_grid, cv = 5)
cv_et_fit = cv_et.fit(X_train, y_train)

#Print the best parameter
print(cv_et_fit.best_params_)

In [ ]:
print('The optimal max_depth for the ExtraTreesRegressor is: {}'.format(cv_et_fit.best_params_['max_depth']))
print('The optimal n_estimators for the ExtraTreesRegressor is: {}'.format(cv_et_fit.best_params_['n_estimators']))

In [ ]:
# Instantiate ETR with optimal hyperparameters
et_basic = RandomForestRegressor(max_depth = 50, n_estimators = 1200)
et_basic.fit(X_train, y_train)
et_basic_pred = et_basic.predict(X_test)

In [ ]:
# Determine R2 score
print('The R2 score for the basic Extra Trees Regressor is {}'.format(r2_score(y_test, et_basic_pred)))

In [ ]:
print('The MSE of the basic ExtraTreesRegressor is {}'.format(mse(y_test, et_basic_pred)))

In [ ]:
# Calculate results
basic_et_r2 = r2_score(y_test, et_basic_pred)
basic_et_mse = mse(y_test, et_basic_pred)
basic_et_rmse = np.sqrt(mse(y_test, et_basic_pred))

In [ ]:
# Create ET DataFrame with dummy regressor results
et_results = pd.DataFrame({'Model':['dummy_reg', 'basic_et'], 'R2': [dummy_r2, basic_et_r2], 'MSE':[dummy_mse, basic_et_mse], 'RMSE':[dummy_rmse, basic_et_rmse]})

In [ ]:
et_results

***

### Using RandomSearchCV to find optimal hyperparameter values

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(30, 140, num = 12)]
max_depth.append(None)
#max_depth.append(9)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10]
# Method of selecting samples for training each tree
bootstrap = [True]

# Create the random grid
random_grid_et = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
et = ExtraTreesRegressor(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
et2 = RandomizedSearchCV(estimator = et, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
et2.fit(X_train, y_train)

In [ ]:
et2.best_params_

In [ ]:
# Instantiate RFR with optimal hyperparameters
et_random = ExtraTreesRegressor(n_estimators= 300,
                             min_samples_split= 2,
                             min_samples_leaf= 1,
                             max_features= 'auto',
                             max_depth= 120,
                             bootstrap= True)
et_random.fit(X_train, y_train)
et_random_pred = et_random.predict(X_test)

In [ ]:
# Determine R2 score
print('The R2 score for the ExtraTreesRegressor with RSCV tuning is {}'.format(r2_score(y_test, et_random_pred)))

In [ ]:
print('The MSE of the ExtraTreesRegressor with RSCV tuning is {}'.format(mse(y_test, et_random_pred)))

In [ ]:
# Add results
et_results.loc[len(et_results)]= ['et_randomCV', 
                                 r2_score(y_test, et_random_pred),
                                 mse(y_test, et_random_pred),
                                 np.sqrt(mse(y_test, et_random_pred))]

In [ ]:
et_results

#### Using GridSearchCV to find the optimal hyperparamter values

In [ ]:
# Create the parameter grid based on the results of random search 
GS_param_grid_et = {
    'bootstrap': [True],
    'max_depth': np.linspace(40, 150, 12),
    'max_features': ['auto'],
    'min_samples_leaf': [2, 3],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [300, 500, 800, 1000, 1200, 1600]
}

In [ ]:
# Create a base model
et = ExtraTreesRegressor()
# Instantiate the grid search model
et3 = GridSearchCV(estimator = et, param_grid = GS_param_grid_et, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
et3.fit(X_train, y_train)

In [ ]:
et3.best_params_

In [ ]:
# Instantiate RFR with optimal hyperparameters
et_grid = ExtraTreesRegressor(n_estimators= 800,
                             min_samples_split= 4,
                             min_samples_leaf= 2,
                             max_features= 'auto',
                             max_depth= 90,
                             bootstrap= True)
et_grid.fit(X_train, y_train)
et_grid_pred = et_grid.predict(X_test)

In [ ]:
# Determine R2 score
print('The R2 score for the Extra Trees Regressor with GSCV is {}'.format(r2_score(y_test, et_grid_pred)))

In [ ]:
print('The MSE of the ExtraTreesRegressor with GSCV is {}'.format(mse(y_test, et_grid_pred)))

In [ ]:
# Add results
et_results.loc[len(et_results)]= ['et_gridCV', 
                                 r2_score(y_test, et_grid_pred),
                                 mse(y_test, et_grid_pred),
                                 np.sqrt(mse(y_test, et_grid_pred))]

In [ ]:
et_results 

### Bayestian Optimization with hyperopt

In [ ]:
trials_et = Trials()

n_estimators-> 50 until 1500, 10
max_depth-> 5, 150 or 200, 10
if hp improves greatly, run similar hp grids on rf and et gridsearch and random search

In [ ]:
space_et = {
    "n_estimators": hp.choice("n_estimators", [400, 500, 600, 700, 800, 900, 1000, 1100, 1200]),
    "max_depth": hp.quniform("max_depth", 30, 120, 10),
}

n_estimators-> start from 50 until 1500, 10
max_depth-> start from 5 until 150 or 200 append None
If better hp, experiment with hp for rf and et

In [ ]:
best_et = fmin(
    fn=hyperparameter_tuning,
    space = space_et, 
    algo=tpe.suggest, 
    max_evals=100, 
    trials=trials_et
)

print("Best: {}".format(best))

In [ ]:
# Instantiate RFR with optimal hyperparameters
et4 = ExtraTreesRegressor(n_estimators= 1000,
                             max_depth= 30)
                             
et4.fit(X_train, y_train)
et_pred4 = et4.predict(X_test)

In [ ]:
# Add results
et_results.loc[len(rf_results)]= ['et_bayes_hyperopt', 
                                 r2_score(y_test, et_pred4),
                                 mse(y_test, et_pred4),
                                 np.sqrt(mse(y_test, et_pred4))]

In [ ]:
et_results

For the Extra Trees Regressor, it seems that the hyperparameters suggested by RandomizedSearchCV (stored in `et_randomCV`) produced the best R2, MSE, and RMSE, so we'll use those hyperparameters for our final Voting Regressor.

## 3. CatBoost Regressor

Now for the third, and final, estimator in our ensemble method, a CatBoost Regressor. First we'll run the regressor without using the Pool function, and then we'll run it with the Pool function (each time testing a range of hyperparameters) to determine whether the model performs better with or without the Pool function. Then, we'll perform Coarse to Fine hyperparameter tuning, using random search and then grid search, just as with the above models. 

### Without Pool

In [ ]:
cat_features_names = ['Month', 'day_of_week', 'state_id_MA', 'state_id_CT', 'state_id_VT', 'state_id_ME'] # here we specify names of categorical features
cat_features = [X.columns.get_loc(col) for col in cat_features_names]
print(cat_features)

In [ ]:
params = {'loss_function':'RMSE',
          'eval_metric':'R2',
          'cat_features': cat_features,
          'verbose': 200,
          'early_stopping_rounds': 200,
          'random_seed': 42
         }
cbc_1 = CatBoostRegressor(**params)
cbc_1.fit(X_train, y_train,
          eval_set=(X_test, y_test),
          use_best_model=True,
          plot=True
         );

In [ ]:
#display.Image("cat_boost_wo_pool.png")

In [ ]:
pred_cbc1= cbc_1.predict(X_test)

In [ ]:
r2_cbc1 = r2_score(y_test, pred_cbc1)
rmse_cbc1 = np.sqrt(mse(y_test, pred_cbc1))
mse_cbc1 = mse(y_test, pred_cbc1)

In [ ]:
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse_cbc1))
print('MSE: {:.2f}'.format(mse_cbc1))
print('R2: {:.2f}'.format(r2_cbc1))

### With Pool

In [ ]:
train_dataset = Pool(X_train, y_train) 
test_dataset = Pool(X_test, y_test)

In [ ]:
model_cbc2 = CatBoostRegressor(loss_function='RMSE', random_seed=42)

In [ ]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

In [ ]:
model_cbc2.grid_search(grid, train_dataset)

In [ ]:
pred_cbc2 = model_cbc2.predict(X_test)
rmse_cbc2 = (np.sqrt(mse(y_test, pred_cbc2)))
r2_cbc2 = r2_score(y_test, pred_cbc2)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse_cbc2))
print('R2: {:.2f}'.format(r2_cbc2))

In [ ]:
model_cbc2_params = {'depth': 8,
  'iterations': 150,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.5}

In [ ]:
print(model_cbc2_params)

It looks like the model performed slightly better when using the Pool function on the training and testing sets, so we'll use the Pool function when conducting Coarse to Fine hyperparameter tuning. We'll use five-fold cross-validation as we did with the previous two estimators.  

### RandomizedSearch with cross-validation

In [ ]:
cbc_random = CatBoostRegressor(loss_function='RMSE', random_seed=42)

In [ ]:
grid_random = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

In [ ]:
cbc_random.randomized_search(grid_random, train_dataset, cv=5)

In [ ]:
pred_cbc_random = cbc_random.predict(X_test)
rmse_cbc_random = (np.sqrt(mse(y_test, pred_cbc_random)))
r2_cbc_random = r2_score(y_test, pred_cbc_random)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse_cbc_random))
print('R2: {:.2f}'.format(r2_cbc_random))

In [ ]:
randomized_cbc_params = {'depth': 8,
  'iterations': 100,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.5},

In [ ]:
print(randomized_cbc_params)

### GridSearch with cross-validation

In [ ]:
cbc_grid = CatBoostRegressor(loss_function='RMSE', random_seed=42)

In [ ]:
grid_cv = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

In [ ]:
cbc_grid.grid_search(grid_cv, train_dataset, cv=5)

In [ ]:
pred_cbc_gridcv = cbc_grid.predict(X_test)
rmse_cbc_gridcv = (np.sqrt(mse(y_test, pred_cbc_gridcv)))
r2_cbc_gridcv = r2_score(y_test, pred_cbc_gridcv)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse_cbc_gridcv))
print('R2: {:.2f}'.format(r2_cbc_gridcv))

In [ ]:
gridcv_cbc_params = {'depth': 8,
  'iterations': 150,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.5}

In [ ]:
print(gridcv_cbc_params)

It looks like RandomizedSearch with cv and GridSearch with cv produced equivalent R2 and RMSE, but used *slightly* different hyperparameters. Because GridSearch with cross-validation is more exhaustive than RandomizedSearch, let's go with GridSearch's optimal hyperparameters for the final ensemble model.

****

### Voting Regressor

Now that we've determined the optimal hyperparameters for each of our three estimators, it's time to pass unfitted versions of these estimators to our final Voting Regressor. 

In [ ]:
# Create the base estimators
# RandomForestRegressor
rf_reg = RandomForestRegressor(bootstrap = True,
                             max_depth = 50,
                             max_features = 'auto',
                             #min_samples_leaf = 2,
                             #min_samples_split = 2,
                             n_estimators = 20,
                             random_state=42
)

# ExtraTreesRegressor
et_reg = ExtraTreesRegressor(n_estimators = 300,
                             min_samples_split = 2,
                             min_samples_leaf = 1,
                             max_features = 'auto',
                             max_depth = 120,
                             bootstrap = True,
                             random_state = 42)

# CatBoostRegressor
cb_reg = CatBoostRegressor(iterations=150,
                           depth = 8,
                           learning_rate = 0.1,
                           l2_leaf_reg = 0.5,
                           random_seed = 42,
                           )
#cb_reg.set_params(**cbc_params)

In [ ]:
# Create the Voting Classifier 

clf_voting = VotingRegressor(estimators = [('rf_est', rf_reg), ('et_est', et_reg),
                                           ('cb_est', cb_reg)])
#Fit and predict

clf_voting.fit(X_train, y_train)
pred_voting = clf_voting.predict(X_test)

In [ ]:
print('Unweighted VotingReg R2: ', r2_score(y_test, pred_voting))
print('Unweighted VotingReg MSE: ', mse(y_test, pred_voting))
print('Unweighted VotingReg RMSE: ', np.sqrt(mse(y_test, pred_voting)))

To determine the optimal weights to use for each of the estimators, we'll use nested for loops to determine the weight combinations that lead to the best evaluation metrics. 

In [ ]:
# Create empty lists that will store the different weights

weights1 = []
weights2 = []
weights3 = []
scores = []

# Create a for loop to evaluate different combinations of weights

for i in np.arange(0.1,1, 0.1):
    for j in np.arange(0.1,1, 0.1):
        for k in np.arange(0.1,1, 0.1):
            reg_voting = VotingRegressor(estimators = [('rf_est', rf_reg), ('et_est', et_reg),
                                           ('cb_est', cb_reg)], weights = [i, j, k])
            reg_voting.fit(X_train, y_train)
            pred = reg_voting.predict(X_test)
            score = r2_score(y_test, pred)
            scores.append(score)
            weights1.append(i)
            weights2.append(j)
            weights3.append(k)

In [ ]:
# Save the results in a data frame

test_scores = pd.DataFrame()
test_scores['Weight1'] = weights1
test_scores['Weight2'] = weights2
test_scores['Weight3'] = weights3
test_scores['Test Score'] = scores

# Create an additional column to save the sum of all the weights

test_scores['sum_weights'] = test_scores['Weight1'].add(test_scores['Weight2']).add(test_scores['Weight3'])

#We are only getting the rows that the sum of all weights were equal to one

condition = test_scores['sum_weights'] == 1

test_scores = test_scores.loc[condition]

In [ ]:
# Sort the values to see the different test scores depending on the weights
test_scores.sort_values(by = 'Test Score', ascending = False).head(15)

It seems that the Voting Regressor performs best with a weight of 0.1 on estimator 1 (Random Forest), a weight of 0.6 on estimator 2 (Extra Trees), and a weight of 0.3 on estimator 3 (CatBoost). Let's apply the weights.

In [ ]:
# Create the Voting Classifier with the most equally weighted because all models performed similarly

reg_voting = VotingRegressor(estimators = [('rf_est', rf_reg), ('et_est', et_reg),
                                           ('cb_est', cb_reg)], weights = [0.1, 0.6, 0.3])

#Fit and predict

reg_voting.fit(X_train, y_train)
pred_voting = reg_voting.predict(X_test)

In [ ]:
R2_final = r2_score(y_test, pred_voting)
MSE_final = mse(y_test, pred_voting)
RMSE_final = np.sqrt(mse(y_test, pred_voting))

In [ ]:
print('Final VotingRegressor R2: ', R2_final)
print('Final VotingRegressor MSE: ', MSE_final)
print('Final VotingRegressor RMSE: ', RMSE_final)

Our model's final R2 is about .86!

## Feature importance

Let's briefly investigate feature importance to see if we find anything unusual. There is currently no way of directly determining feature importance of a Voting Regressor, so instead, we'll define a function, `compute_feature_importance`, to determine the feature importances of each of the three individual estimators fed to the final Voting Regressor, and then apply the same weights to the feature importances as we did to the estimators themselves above. 

In [ ]:
def compute_feature_importance(voting_clf, weights):
    """ Function to compute feature importance of Voting Classifier """
    
    feature_importance = {}
    for est in reg_voting.estimators_:
        feature_importance[str(est)] = est.feature_importances_
    
    fe_scores = [0]*len(list(feature_importance.values())[0])
    for idx, imp_score in enumerate(feature_importance.values()):
        imp_score_with_weight = imp_score*weights[idx]
        fe_scores = list(np.add(fe_scores, list(imp_score_with_weight)))
    return fe_scores

In [ ]:
feat_df = pd.DataFrame()
feat_df['Feature'] = X_train.columns
feat_df['Feature Importance'] = compute_feature_importance(reg_voting, [0.1, 0.6, 0.3])
feat_df=(feat_df.sort_values('Feature Importance', ascending = False)).reset_index()

In [ ]:
feat_df = feat_df.drop(columns='index')

In [ ]:
feat_df

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
sns.set_style("ticks")
fig, ax = plt.subplots()
plt.barh(feat_df['Feature'],
         feat_df['Feature Importance'])
plt.xlabel('Feature Importance', fontsize=16)
plt.ylabel('Feature', fontsize=16)
#ax.set_ylabel('Feature')
plt.title('Feature Importances in Weighted Voting Regressor', fontsize=18)
sns.despine()
plt.show()